In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from __future__ import print_function
import keras
from keras.utils import to_categorical
import os
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
from keras import optimizers, losses, activations, models
from keras.layers import Convolution2D, Dense, Input, Flatten, Dropout, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras import applications
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = False,
                                  validation_split = 0.05)

#test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('data/asl_alphabet_train/',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical',
                                                subset = 'training')
# to save the augmented images run:
# for inputs, outputs in training_set:
#     pass
    
test_set = train_datagen.flow_from_directory('data/asl_alphabet_train/',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode = 'categorical',
                                           subset = 'validation')

In [9]:
input_shape = (150, 150, 3)
nclass = 29

base_model = applications.InceptionV3(weights='imagenet', 
                                include_top=False, 
                                input_shape=(150, 150, 3))
base_model.trainable = False

add_model = Sequential()
add_model.add(base_model)
add_model.add(GlobalAveragePooling2D())
add_model.add(Dropout(0.5))
add_model.add(Dense(29, activation='softmax'))
model = add_model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d_4 ( (None, 2048)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 29)                59421     
Total params: 21,862,205
Trainable params: 59,421
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
history = model.fit_generator(training_set,
                              epochs=10,
                              validation_data = test_set)

## Saving the Model

In [ ]:
# Save the weights
model.save_weights('models/transfer_learning/model_weights.h5')

# Save the model architecture
with open('models/transfer_learning/model_architecture.json', 'w') as f:
    f.write(model.to_json())

## Graph the accuracy and loss

In [ ]:
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Loading the Model

In [ ]:
from keras.models import model_from_json

json_file = open('models/cnn/model_architecture.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# loading weights
loaded_model.load_weights("models/cnn/model_weights.h5")

loaded_model.compile(optimizer = 'adam', loss='categorical_crossentropy',metrics=['accuracy'])
loaded_model.summary()

## Visualization of the network

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='resources/transfer_model.png')